In [ ]:
!pip install pytorch_lightning 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pytorch_lightning as pl
from tqdm.notebook import tqdm

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/USML_Data/movies.csv')

In [ ]:
import pickle
class Checkpoint():
    
    def save_params(self,test,train,movie_ids,ratings,items,users,model_name = ""):
        with open(r"/content/drive/MyDrive/USML_Data/NCF/model_data" + model_name, 'wb') as fp:
            pickle.dump([test,train,movie_ids,ratings,items,users], fp)
        return

    def load_params(self,model_name):
        with open(r"/content/drive/MyDrive/USML_Data/NCF/model_data" + model_name, 'rb') as fp:
            test,train,movie_ids,ratings,items,users = pickle.load(fp)
        return test,train,movie_ids,ratings,items,users
    
    def saveModel(self,model,model_name):
        torch.save(model.state_dict(), r"/content/drive/MyDrive/USML_Data/NCF/model_data" + model_name + ".pt")
        return model
    
    def loadModel(self,model_name, model = None):
        state_dict = torch.load(r"/content/drive/MyDrive/USML_Data/NCF/model_data" + model_name + ".pt")
        model.load_state_dict(state_dict)
        return model

In [ ]:
class Net(pl.LightningModule):

    def __init__(self,users,items,movie_ids,ratings, pr = None):
        super().__init__()
        self.user_embedding = nn.Embedding(users, 8)
        self.item_embedding = nn.Embedding(items, 8)
        self.hidden1 = nn.Linear(16,64)
        self.hidden2 = nn.Linear(64,32)
        self.output = nn.Linear(32,1)
        self.ratings = ratings
        self.movie_ids = movie_ids
        self.pr = pr

    def forward(self, user_list, item_list):
        user_out = self.user_embedding(user_list)
        item_out = self.item_embedding(item_list)
        combined = torch.cat([user_out,item_out],dim= -1)
        combined = nn.ReLU()(self.hidden1(combined))
        combined = nn.ReLU()( self.hidden2(combined))
        pred = nn.Sigmoid()(self.output(combined))
        
        return pred
    
    def training_step(self, batch, batch_idx):
        user_list, item_list, label_list = batch
        y_hat = self(user_list,item_list)
        # BCE: Also called Softmax Loss. It is a Softmax activation plus a Cross-Entropy loss
        loss = nn.BCELoss()(y_hat, label_list.view(-1, 1).float())
        return loss
    
    def configure_optimizers(self):
        # adam optimizer with lr = 0.001
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def train_dataloader(self):
        return DataLoader(DataWrapper(self.ratings, self.movie_ids),
                          batch_size=512, num_workers=4)




In [ ]:
def reccomend_user_movies(u,ratings,movie_ids,net,movies):
    print("Showing recommendations for user: {}".format(u))
    print("====" * 9)
    print("Movies with high ratings from user")
    print("----" * 8)

    user_interacted_items = ratings.groupby('userId')['movieId'].apply(list).to_dict()
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(movie_ids) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 100))    
    predicted_labels = np.squeeze(net(torch.tensor([u]*100), torch.tensor(selected_not_interacted)).detach().numpy())
    labels = np.argsort(predicted_labels)[::-1][0:10].tolist()
    top10_items = [selected_not_interacted[i] for i in labels]
    user_top_ratings = ratings[ratings["movieId"].isin(user_interacted_items[u])].sort_values('rating', ascending = False).head().movieId
    movie_df_rows = movies[movies["movieId"].isin(user_top_ratings)]
    for row in movie_df_rows.itertuples():
        print(row.title, ":", row.genres)

    print("----" * 8)
    print("Top 10 movie recommendations")
    print("----" * 8)
    recommended_movies = movies[movies["movieId"].isin(top10_items)]
    for row in recommended_movies.itertuples():
        print(row.title, ":", row.genres)

    return

In [ ]:
cp = Checkpoint()
model_name = "NCF_70"
test,train,movie_ids,ratings,items,users = cp.load_params(model_name)
net = Net(users,items,movie_ids,ratings)
net = cp.loadModel(model_name,net)
reccomend_user_movies(3454,ratings,movie_ids,net,movies)